# EDA on Wuhan Blod Samples dataset 
... and (hopefuly) more (in the future)

Data visualisation course @ PUT, 04.2021 <br>
by: <br>
Jędrzej Kopiszka <br>
Konrad Szewczyk

In [1]:
import pandas as pd
import pandas_profiling
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

## 1. Data acquisition and transformation

In [2]:
wuhan_blod_samples = pd.read_excel("blod_samples.xlsx")

In [3]:
wuhan_blod_samples.head()

,PATIENT_ID,RE_DATE,age,gender,Admission time,Discharge time,outcome,Hypersensitive cardiac troponinI,hemoglobin,Serum chloride,...,mean corpuscular hemoglobin,Activation of partial thromboplastin time,High sensitivity C-reactive protein,HIV antibody quantification,serum sodium,thrombocytocrit,ESR,glutamic-pyruvic transaminase,eGFR,creatinine
0,1.0,2020-01-31 01:09:00,73,1,2020-01-30 22:12:47,2020-02-17 12:40:09,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,2020-01-31 01:25:00,73,1,2020-01-30 22:12:47,2020-02-17 12:40:09,0,NaN,136.0,NaN,...,31.9,NaN,NaN,NaN,NaN,0.12,NaN,NaN,NaN,NaN
2,NaN,2020-01-31 01:44:00,73,1,2020-01-30 22:12:47,2020-02-17 12:40:09,0,NaN,NaN,103.1,...,NaN,NaN,43.1,NaN,137.7,NaN,NaN,16.0,46.6,130.0
3,NaN,2020-01-31 01:45:00,73,1,2020-01-30 22:12:47,2020-02-17 12:40:09,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,2020-01-31 01:56:00,73,1,2020-01-30 22:12:47,2020-02-17 12:40:09,0,19.9,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
wuhan_blod_samples.outcome = wuhan_blod_samples.outcome.astype('category')
wuhan_blod_samples.gender = wuhan_blod_samples.gender.astype('category')

In [5]:
refactored_columns = []
for column in wuhan_blod_samples.columns:
    column = column.lower()
    column = column.replace(" ", "_").replace("-", "_").replace("(", "_")
    column = column.strip("(%)").strip("(#").strip("β").strip("α").strip("_").strip("%)")
    refactored_columns.append(column)

wuhan_blod_samples.columns = refactored_columns

In [6]:
wuhan_blod_samples.columns

Index(['patient_id', 're_date', 'age', 'gender', 'admission_time',
       'discharge_time', 'outcome', 'hypersensitive_cardiac_troponini',
       'hemoglobin', 'serum_chloride', 'prothrombin_time', 'procalcitonin',
       'eosinophils', 'interleukin_2_receptor', 'alkaline_phosphatase',
       'albumin', 'basophil', 'interleukin_10', 'total_bilirubin',
       'platelet_count', 'monocytes', 'antithrombin', 'interleukin_8',
       'indirect_bilirubin', 'red_blood_cell_distribution_width',
       'neutrophils', 'total_protein',
       'quantification_of_treponema_pallidum_antibodies',
       'prothrombin_activity', 'hbsag', 'mean_corpuscular_volume',
       'hematocrit', 'white_blood_cell_count', 'tumor_necrosis_factor',
       'mean_corpuscular_hemoglobin_concentration', 'fibrinogen',
       'interleukin_1', 'urea', 'lymphocyte_count', 'ph_value',
       'red_blood_cell_count', 'eosinophil_count', 'corrected_calcium',
       'serum_potassium', 'glucose', 'neutrophils_count', 'direct_bilir

In [7]:
wuhan_blod_samples.drop(labels=["2019_ncov_nucleic_acid_detection"], axis=1, inplace=True)

In [8]:
for row, content in enumerate(wuhan_blod_samples.patient_id):
    if pd.isna(content):
        wuhan_blod_samples.patient_id[row] = wuhan_blod_samples.patient_id[row-1]

<ipython-input-8-839919343246>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wuhan_blod_samples.patient_id[row] = wuhan_blod_samples.patient_id[row-1]


In [9]:
wuhan_blod_samples.patient_id = wuhan_blod_samples.patient_id.astype("int64")

In [10]:
def missing_value_imputer(column, data):
    current_patient = 1
    upper_limit = None
    lower_limit = None
    missing_rows_tab = []
    for row_index, row_content in enumerate(data.iterrows()):
        if row_content[1].patient_id == current_patient:
            if pd.isna(row_content[1][column]) and upper_limit is None:
                missing_rows_tab.append(row_index)

            elif upper_limit is None and pd.isna(row_content[1][column]) is False:
                if len(missing_rows_tab):
                    data.loc[min(missing_rows_tab): max(missing_rows_tab)+1, column] = row_content[1][column]
                upper_limit = row_content[1][column]
                lower_limit = None
                missing_rows_tab = []

            elif pd.isna(row_content[1][column]) is False and lower_limit is None:
                lower_limit = row_content[1][column]
                iteration = 1
                difference = (lower_limit - upper_limit) / (len(missing_rows_tab) + 1)
                for row_y in missing_rows_tab:
                    data.loc[row_y, column] = upper_limit + (difference * iteration)
                    iteration += 1

                upper_limit = row_content[1][column]
                lower_limit = None
                missing_rows_tab = []

            elif lower_limit is None:
                missing_rows_tab.append(row_index)

        else:
            for row_y in missing_rows_tab:
                data.loc[row_y, column] = upper_limit

            current_patient = row_content[1].patient_id
            upper_limit = None
            lower_limit = None
            missing_rows_tab = []

            if pd.isna(row_content[1][column]):
                missing_rows_tab.append(row_index)

            else:
                upper_limit = row_content[1][column]
                
    if len(missing_rows_tab):
        if upper_limit is not None:
            for row_y in missing_rows_tab:
                data.loc[row_y, column] = upper_limit
        else:
            for row_y in missing_rows_tab:
                data.loc[row_y, column] = lower_limit
                
    return data

In [11]:
from copy import deepcopy
wuhan_blod_samples_imputed = deepcopy(wuhan_blod_samples)

In [12]:
for column in wuhan_blod_samples_imputed.columns[7:]:
    wuhan_blod_samples_imputed = missing_value_imputer(column, wuhan_blod_samples_imputed)

In [13]:
wuhan_blod_samples_imputed.eosinophils[wuhan_blod_samples_imputed.eosinophils.isna()]

737    NaN
738    NaN
739    NaN
740    NaN
741    NaN
3068   NaN
3076   NaN
3097   NaN
3179   NaN
3213   NaN
3214   NaN
4184   NaN
4185   NaN
4186   NaN
4187   NaN
4188   NaN
4189   NaN
4190   NaN
4191   NaN
4192   NaN
4297   NaN
4539   NaN
4575   NaN
4592   NaN
4593   NaN
4594   NaN
4595   NaN
4643   NaN
4799   NaN
4857   NaN
5213   NaN
5705   NaN
5776   NaN
5839   NaN
Name: eosinophils, dtype: float64

In [14]:
wuhan_blod_samples_imputed.eosinophils.isna().sum()

34

In [15]:
for column in wuhan_blod_samples_imputed.isna().sum().index:
    print("{} has {} NaNs".format(column, wuhan_blod_samples_imputed.isna().sum().loc[column]))

patient_id has 0 NaNs
re_date has 14 NaNs
age has 0 NaNs
gender has 0 NaNs
admission_time has 0 NaNs
discharge_time has 0 NaNs
outcome has 0 NaNs
hypersensitive_cardiac_troponini has 1022 NaNs
hemoglobin has 34 NaNs
serum_chloride has 46 NaNs
prothrombin_time has 79 NaNs
procalcitonin has 474 NaNs
eosinophils has 34 NaNs
interleukin_2_receptor has 1871 NaNs
alkaline_phosphatase has 32 NaNs
albumin has 32 NaNs
basophil has 34 NaNs
interleukin_10 has 1884 NaNs
total_bilirubin has 32 NaNs
platelet_count has 34 NaNs
monocytes has 34 NaNs
antithrombin has 2280 NaNs
interleukin_8 has 1871 NaNs
indirect_bilirubin has 49 NaNs
red_blood_cell_distribution_width has 114 NaNs
neutrophils has 34 NaNs
total_protein has 32 NaNs
quantification_of_treponema_pallidum_antibodies has 1141 NaNs
prothrombin_activity has 79 NaNs
hbsag has 1131 NaNs
mean_corpuscular_volume has 34 NaNs
hematocrit has 34 NaNs
white_blood_cell_count has 30 NaNs
tumor_necrosis_factor has 1871 NaNs
mean_corpuscular_hemoglobin_conc

In [16]:
prof_sample = wuhan_blod_samples_imputed.sample(frac=0.3)

prof = pandas_profiling.ProfileReport(prof_sample, minimal=True)
prof.to_file("WuhanBlodSample_EDA_imputed_sample.html")

In [18]:
wuhan_blod_samples_imputed.head(5)

,patient_id,re_date,age,gender,admission_time,discharge_time,outcome,hypersensitive_cardiac_troponini,hemoglobin,serum_chloride,...,mean_corpuscular_hemoglobin,activation_of_partial_thromboplastin_time,high_sensitivity_c_reactive_protein,hiv_antibody_quantification,serum_sodium,thrombocytocrit,esr,glutamic_pyruvic_transaminase,egfr,creatinine
0,1,2020-01-31 01:09:00,73,1,2020-01-30 22:12:47,2020-02-17 12:40:09,0,19.9,136.0,103.1000,...,31.900,37.9,43.1000,0.09,137.70,0.12000,41.0,16.00,46.6000,130.0
1,1,2020-01-31 01:25:00,73,1,2020-01-30 22:12:47,2020-02-17 12:40:09,0,19.9,136.0,103.1000,...,31.900,37.9,43.1000,0.09,137.70,0.12000,41.0,16.00,46.6000,130.0
2,1,2020-01-31 01:44:00,73,1,2020-01-30 22:12:47,2020-02-17 12:40:09,0,19.9,136.5,103.1000,...,31.925,37.9,43.1000,0.09,137.70,0.13375,41.0,16.00,46.6000,130.0
3,1,2020-01-31 01:45:00,73,1,2020-01-30 22:12:47,2020-02-17 12:40:09,0,19.9,137.0,102.8875,...,31.950,37.9,38.1625,0.09,138.35,0.14750,41.0,19.25,49.8625,125.0
4,1,2020-01-31 01:56:00,73,1,2020-01-30 22:12:47,2020-02-17 12:40:09,0,19.9,137.5,102.6750,...,31.975,37.9,33.2250,0.09,139.00,0.16125,41.0,22.50,53.1250,120.0


In [ ]:
wuhan_blod_samples_imputed.hemoglobin